# Tiền xử lí dữ liệu

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## I. Chuẩn bị dữ liệu để xử lí

### I.1 Lấy dữ liệu thô


In [11]:
csv_path = '../data/BankChurners.csv'
data = np.genfromtxt(csv_path, delimiter=',', dtype=str, encoding='ascii')

# Loại bỏ hai cột cuối không cần thiết
data = data[:, :-2]
columns = data[0, :]
columns = np.char.replace(columns, '"', '')
data = data[1:, :]
data = np.char.replace(data, '"', '')

### I.2 Tạo từ điển để ánh xạ nhanh tên cột với dữ liệu của cột

In [12]:
# Tạo một từ điển để ánh xạ tên cột với chỉ số của chúng
dict_data = {'CLIENTNUM': 0, 
             'Attrition_Flag': 1,
             'Customer_Age': 2,
             'Gender': 3,
             'Dependent_count': 4, 
             'Education_Level': 5, 
             'Marital_Status': 6,
             'Income_Category': 7, 
             'Card_Category': 8, 
             'Months_on_book': 9,
             'Total_Relationship_Count': 10, 
             'Months_Inactive_12_mon': 11, 
             'Contacts_Count_12_mon': 12,
             'Credit_Limit': 13, 
             'Total_Revolving_Bal': 14, 
             'Avg_Open_To_Buy': 15, 
             'Total_Amt_Chng_Q4_Q1': 16,
             'Total_Trans_Amt': 17, 
             'Total_Trans_Ct': 18, 
             'Total_Ct_Chng_Q4_Q1': 19, 
             'Avg_Utilization_Ratio': 20}

## II. Bỏ những cột không quan trọng trong việc thiết kế mô hình
Trong quá trình phân tích dữ lệu thì ta sẽ không sử dụng đến một dữ liệu với những lí do sau:
1.  **`Avg_Open_To_Buy`**: Vì có tương quan tương quan tuyệt đối **1.00** với `Credit_Limit`. Về mặt toán học, `Avg_Open_To_Buy = Credit_Limit - Revolving_Bal`. Việc giữ cả hai không mang lại thêm thông tin mà gây nhiễu và lãng phí tài nguyên tính toán.
2.  **`Gender`**: Trong biểu đồ tần suất được so với `Attrition_Flag`, cột định dang này có tần suất khá giống nhau ở cả hai giá trị M (nam giới) và F (nữ giới) và cũng có tương quan Cramér's V khá thấp với `Attriton_Flag`(**0.01**)

## III. Chuẩn hóa cột đặc trưng Attrition_Flag và các cột định danh


- Ta sẽ xử lí một số cột danh trước khi chuẩn hóa: 
1.  **`Card_Category`**: Vì số lượng thẻ `Blue` chiếm phần lớn nên ta sẽ gộp nhóm (Binning) tất cả các thẻ còn lại (`Sliver`, `Gold`, `Plantinum`) và chuyển hóa thành `0` (Không phải thẻ Blue) và `1`(Có thẻ Blue).
2. **`Marital status`**: Ta sẽ gộp 2 cột `Divorced` và `Single` vì xét theo logic thì nó vẫn có chung một ý nghĩa là hiện tại vẫn chưa có hôn nhân.
- Sau đó ta sẽ áp dụng các thuật toán để chuẩn hóa các cột:
1. `One-hot encoding` cho các cột định danh có giá trị Unknown (Vì giá trị này vẫn có tỉ lệ từ `7 - 15%` trong các cột) gồm `Education_Level, Income_Category và Marital_Status`
2. `Binary encoding` cho cột `Attrition_Flag và Card_Category`

In [13]:
def one_hot_np(col):
    uniq = np.unique(col)
    return (col[:, None] == uniq).astype(np.int8), uniq

card_col = data[:, dict_data['Card_Category']]
card_col = (card_col == 'Blue').astype(np.int8)

attrition_col = data[:, dict_data['Attrition_Flag']]
attrition_col = (attrition_col == 'Existing Customer').astype(np.int8)

marital_col = data[:, dict_data['Marital_Status']]
mapped_marital = np.where(
    np.isin(marital_col, ["Divorced", "Single"]), 
    "NotMarried", 
    marital_col
)
marital_ohe, marital_uniques = one_hot_np(mapped_marital)

education_col = data[:, dict_data['Education_Level']]
education_ohe, education_uniques = one_hot_np(education_col)

income_col = data[:, dict_data['Income_Category']]
income_ohe, income_uniques = one_hot_np(income_col)


## IV. Điều chỉnh kích thước cho các cột có giá trị số

### IV.1. Áp dụng Z-score Normalization cho nhóm biến phân phối chuẩn

Đối với những cột dữ liệu có phân phối tương đối ổn định, ít bị lệch (non-skewed) hoặc có dạng gần với phân phối chuẩn (Gaussian), ta áp dụng phương pháp chuẩn hóa Z-score (Standardization). Phương pháp này giúp đưa các biến về cùng một thang đo thống kê chung, loại bỏ sự khác biệt về đơn vị đo lường (scale invariance) giữa các đặc trưng:

* `Customer_Age`
* `Dependent_count`
* `Total_Relationship_Count`
* `Months_Inactive_12_mon`
* `Contacts_Count_12_mon`
* `Total_Amt_Chng_Q4_Q1`
* `Total_Ct_Chng_Q4_Q1`
* `Months_on_book`

**Cơ sở toán học:**

Giá trị được chuyển đổi bằng cách lấy hiệu số giữa giá trị gốc và giá trị trung bình, sau đó chia cho độ lệch chuẩn. Quá trình này tập trung dữ liệu về gốc tọa độ 0 và co giãn độ phân tán sao cho độ lệch chuẩn bằng 1.

$$x_{scaled} = \frac{x - \mu}{\sigma}$$

Trong đó:
* $x$: Giá trị gốc ban đầu.
* $\mu$: Giá trị trung bình (mean) của toàn bộ cột dữ liệu.
* $\sigma$: Độ lệch chuẩn (standard deviation) của toàn bộ cột dữ liệu.
* $x_{scaled}$: Giá trị sau khi đã được chuẩn hóa.

In [14]:
numerical_score = [
    'Customer_Age', 'Dependent_count', 'Total_Relationship_Count',
    'Months_Inactive_12_mon', 'Contacts_Count_12_mon',
    'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1',
    'Months_on_book'
]

def zscore(x):
    return (x - x.mean()) / x.std()

numerical_score_col = np.empty((data.shape[0], 0))

for col in numerical_score:
    col_data = data[:, dict_data[col]].astype(float)
    col_z = zscore(col_data)
    numerical_score_col = np.hstack((numerical_score_col, col_z.reshape(-1, 1)))

### Tạo cột mới: Avg_Total_Trans

```
Avg_Total_Trans = Total_Trans_Amt / Total_Trans_Ct
```


In [15]:
trans_amt = data[:, dict_data['Total_Trans_Amt']].astype(float)
trans_ct = data[:, dict_data['Total_Trans_Ct']].astype(float)
average_total = trans_amt / trans_ct

### IV.2. Áp dụng Log Scaling cho nhóm biến phân phối lệch phải, có giá trị lớn

Đối với những cột dữ liệu có dải giá trị rộng và biểu đồ phân phối bị lệch hẳn về phía bên phải (right-skewed), ta áp dụng phương pháp biến đổi Logarit. Phương pháp này giúp co hẹp khoảng giá trị, đưa phân phối về dạng cân đối hơn và giảm thiểu tác động của các giá trị ngoại lai (outliers):

* `Credit_Limit`
* `Avg_Open_To_Buy`
* `Total_Revolving_Bal`
* `Avg_Total_Trans` (cột mới tạo)

**Cơ sở toán học:**

Giá trị được chuyển đổi bằng cách lấy logarit tự nhiên của (giá trị gốc + 1). Việc cộng thêm 1 vào công thức là bắt buộc để đảm bảo tính toán học trong trường hợp dữ liệu đầu vào có giá trị bằng 0 (vì logarit của 0 không xác định).

$$x_{log} = \ln(1 + x)$$

Trong đó:
* $x$: Giá trị gốc ban đầu.
* $1$: Hằng số cộng thêm để tránh lỗi toán học khi $x = 0$.
* $\ln$: Logarit tự nhiên (cơ số $e$).

In [16]:

numerical_log = [
    'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy',
    'Avg_Total_Trans']

def log_scale(x):
    return np.log1p(x)

numerical_log_col = np.empty((data.shape[0], 0))
for col in numerical_log:
    if col == 'Avg_Total_Trans':
        col_data = average_total
    else:
        col_data = data[:, dict_data[col]].astype(float)
    col_log = log_scale(col_data)
    numerical_log_col = np.hstack((numerical_log_col, col_log.reshape(-1, 1)))


### IV.3. Sử dụng Min-Max Scaling cho biến có phạm vi [0, 1]

Đối với cột dữ liệu đã có dải giá trị nằm sẵn trong khoảng từ 0 đến 1 nhưng phân phối không đều (thiên lệch về 0), ta sử dụng phương pháp Min-Max Scaling. Việc này nhằm đảm bảo tính nhất quán tuyệt đối về biên độ dữ liệu với các biến khác sau khi chuẩn hóa, giữ nguyên tỉ lệ khoảng cách giữa các giá trị:

* `Avg_Utilization_Ratio` (Giá trị từ 0 đến 1, phân phối lệch về 0)

**Cơ sở toán học:**

Phép biến đổi tuyến tính giúp quy đổi dữ liệu về đoạn [0, 1] dựa trên hai giá trị cực trị (nhỏ nhất và lớn nhất) của tập dữ liệu:

$$x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$$

Trong đó:
* $x$: Giá trị gốc ban đầu.
* $x_{min}$: Giá trị nhỏ nhất của cột dữ liệu.
* $x_{max}$: Giá trị lớn nhất của cột dữ liệu.

In [17]:
def minmax(x):
    return (x - x.min()) / (x.max() - x.min())

util_col = data[:, dict_data['Avg_Utilization_Ratio']].astype(float)
util_mm = minmax(util_col)

## V. Lưu thành các cột đã xử lí vào file mới

In [18]:

final_matrix = np.column_stack([
    attrition_col,          # Target
    card_col,               # Binary Card
    marital_ohe,            # One hot
    education_ohe,
    income_ohe,
    numerical_score_col,                 # Z-score
    numerical_log_col,               # Log-scaled
    util_mm                 # MinMax
])

headers = []

headers.append("Attrition_Flag")
headers.append("Is_Blue_Card")

headers += [f"Marital_{u}" for u in marital_uniques]
headers += [f"Education_{u}" for u in education_uniques]
headers += [f"Income_{u}" for u in income_uniques]

headers += [f"{c}_zscore" for c in numerical_score]
headers += [f"{c}_log" for c in numerical_log]

headers.append("Avg_Utilization_Ratio_minmax")

header_line = ",".join(headers)
np.savetxt(
    "../data/BankChurners_preprocessed.csv",
    final_matrix,
    delimiter=",",
    header=header_line,
    comments=""
)